# Classificação Numérica - Rede Neural

In [ ]:
# instalando bibliotecas
%pip install torch
%pip install torchvision
%pip install matplotlib
%pip install numpy

In [ ]:
# importando bibliotecas
import numpy as pd
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt 
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

## Convertendo dados - Tensor

Tensor é como uma matriz, mas não existe um limite de dimensões, podendo estar entre 0 a n. A biblioteca transforma os dados em tensor, por isso iremos realizar a transformação

In [ ]:

import torch
from torchvision import datasets, transforms
# definindo a imagem para tensor
transform = transforms.ToTensor()

# treino do dataset
trainset = datasets.MNIST('./MNIST_data/', download=True, train=True, transform=transform)
# pegar os dados por parte
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# teste do dataset
testset = datasets.MNIST('./MNIST_data/', download=True, train=False, transform=transform)
# pegar os dados por parte
vallloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

## Conferindo dados

In [ ]:
# abrir um dos itens do dataset para visualizar formato
dataiter = iter(trainloader)
img, target = next(dataiter)
plt.imshow(img[0].numpy().squeeze(), cmap='gray_r')


In [ ]:
# analisando a dimensao do tensor imagem e tensor etiqueta
# 1 = intensidade do preto em cada pixel
# 28 = altura da imagem
# 28 = largura da imagem
print(img[0].shape)

# etique nao possui dimensao por ser um escalar
print(target[0].shape)

## Adicionando elementos da RN

Usaremos 3 funções de ativação: ReLU, ReLU e SoftMax

In [ ]:
class Modelo(nn.Module):
    def __init__(self): 
        super(Modelo, self).__init__()
        ## camada de entrada, 784 neurônios que se ligam a 128
        self.linear1 = nn.Linear(784, 128)
        ## camada oculta 1, 128 neurônios que se ligam a 64
        self.linear2 = nn.Linear(128, 64)
        ## camada oculta 2, 64 neurônios que se ligam a 10
        self.linear3 = nn.Linear(64, 10)

        # nao definimos a camada de saida pois pegamos ela a partir da camada 2

    def forward(self, X):
        # camada de entrada 
        X = F.relu(self.linear1(X))
        # camada oculta 1
        X = F.relu(self.linear2(X))
        # camada oculta 2
        X = self.linear3(X)
        # camada de saida
        return F.log_softmax(X, dim=1)


## Estrutura de treino do modelo

- Calcular a perda a partir da comparação entre as predições e as etiquetas(target) do subgrupo sendo analisado
- Com a perda, calcular o gradiente em relação aos pesos e as bias
- A partir do gradiente e de uma politica de otimização, atualizar os pesos e as bias 

### Otimizador

A partir do optim é possível escolher qual otimizador usar para fazer as atualizações dos pesos e do bias.

In [ ]:
# definindo funcao treino
# interacao trainloader, back propagation e otimizador step sao repetidos ate que todo trainset seja percorrido   
def treino(modelo, trainloader, device):
    # definindo a funcao de perda e o otimizador do peso e do bias
    otimizador = optim.SGD(modelo.parameters(), lr=0.01, momentum=0.5)
    inicio = time()

# definindo criterio para calcular a perda
    criterio = nn.NLLLoss()
    # numero de EPOCHS para treinar
    EPOCHS = 30
    # ativando treinamento do modelo
    modelo.train()

    for epoch in range(EPOCHS):
        perda_acumulada = 0

        for img, target in trainloader:
            # convertendo as img para "vetores" 28*28 para serem compativeis com a entrada
            img = img.view(img.shape[0], -1)
            # zerando o gradiente para o proximo ciclo
            otimizador.zero_grad()

            # colocanndo os dados no modelo
            output = modelo(img.to(device))
            #calculando a perda da epoch atual
            perda_instantanea = criterio(output, target.to(device))

            # back propagation a partir da perda da epoch atual
            perda_instantanea.backward()
            # atualizando pesos e bias
            otimizador.step()
            #atualizacao da perda acumulada
            perda_acumulada += perda_instantanea.item()
        else:
            print("Epoch {} - Perda resultante: {}".format(epoch+1, perda_acumulada/len(trainloader)))
            print("\nTempo de treino (em minutos) =",(time()-inicio)/60)

In [ ]:
# funcão de otimizacao do modelo para teste
def validacao(modelo, valloader, device):
    contas_corretas, conta_todas = 0, 0
    for img, target in valloader:
        for i in range(len(target)):
            imgs = img[i].view(1, 784)
            # desativer autograd. Grafos computacionais dinâmicos tem um custo alto de processamento
            with torch.no_grad():
                # saida do modelo em escala logaritmica
                logps = modelo(imgs.to(device))

            # converte saida escala normal (tensor)
            ps = torch.exp(logps)   
            # probabilidade de cada classe
            probab = list(ps.cpu().numpy()[0])
            # converte o tensor em numero,  o numero que previu correto
            pred_label = probab.index(max(probab))
            pred_certa = target.numpy()[i]
            # compara se o numero previsto é igual ao numero real
            if(pred_certa  == pred_label):
                contas_corretas += 1
            conta_todas += 1

    print("Numero de imagens testadas =", conta_todas)
    print("\nNumero de imagens previstas corretamente =", contas_corretas)
    print("\nAcuracia do modelo =", (contas_corretas*100)/conta_todas, "%")

## Rodando Rede Neural

In [ ]:
# inicializando o modelo
modelo = Modelo()
# definindo o dispositivo de processamento
# modelo rodara na GPU se possivel, caso contrario, rodara na CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo.to(device)

In [ ]:
# treinando o modelo
treino(modelo, trainloader, device)

In [ ]:
# validando o modelo
validacao(modelo, vallloader, device)

